In [2]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [3]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = np.pi * random()
        speeds[i][0] = speed * np.cos(theta)
        speeds[i][1] = speed * np.sin(theta)

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2)))
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repultion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        ##ATTENTION HERE!!
#        if not virtuals[j]:
 #           virtuals[j] = agents[j]
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals

        
#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return


In [35]:
#COUZIN MODEL IMPLEMENTED WITH REPULSE, ATRACT AND ORIENT ZONES SEPARATED (1ST PAPER)
def couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_range, model2, roa, atract, orient):
    
    if not model2:
        atract = orient = 1
    # watch only particles in repulsion zone
    for i in range(N):
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):     
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], virtuals[j])
            
            if in_sight_range(rel_pos, speeds[i], sight_range):
                if distance < rr:
                    rel_pos = normalized(rel_pos)
                    r_dir = r_dir + rel_pos
                    repulsion_flag = True

                elif not repulsion_flag:
                    
                    if model2: # Couzin 2
                        if distance < roa:
                            o_dir = o_dir + speeds[j]
                            rel_pos = normalized(rel_pos)
                            a_dir = a_dir + rel_pos

                    else: # Couzin 1
                        if distance < ro:
                            o_dir = o_dir + speeds[j]

                        elif distance < ra:
                            rel_pos = normalized(rel_pos)
                            a_dir = a_dir + rel_pos

        #Out of for (j), we treat now the resulting direction vector
        if repulsion_flag:
            tot_dir =  normalized(- r_dir)
        else:
            tot_dir = orient * normalized(o_dir) + atract * normalized(a_dir)
            tot_dir = normalized(tot_dir)
            
        tot_dir = normalized(normalized(tot_dir) + noisy_vector(noise))
        
        #avoid pts stoping when not interacting
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = softened_angle(speeds[i], tot_dir, s, dTheta)
    return

    
def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    for i in range(N):
        distances = get_distances(agents[i], agents)
        tot_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                tot_dir = tot_dir + speeds[j]

        tot_dir = s * normalized(normalized(tot_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = tot_dir
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], virtuals[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (-clr*exp(-distance / lr) + cla *exp(- distance / la))
            
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        friction =  beta * ((normalized(speeds[i]))**2) * speeds[i] #friction force propto beta
        d_speed = propulsion - friction - grad_U
        speeds[i]= speeds[i]+dt*d_speed
        print speeds[i], propulsiongrad_U
    return

    
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        #mill (dt, agents, speeds, N, width, height, cr=1.0, ca=0.5, lr=0.5, la=3.0, alpha=1.6, beta=0.5)
        mill (dt, agents, speeds, N, width, height, cr=1.0, ca=0.5, lr=1.5, la=4.0, alpha=1.6, beta=2.5)
        
        
        #interaction(agents, speeds)
        
        
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        #INFORMATION TRANSFER: SHAPE & DIRECTION & ALIGNMENT QUALITY 
        #point_cm.undraw()
        #old_cm = quality(agents, speeds, N, bias, window, old_cm)
        
        ## BOUNDARY CONDITIONS
        #rigid_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return


In [36]:
run(3000, 10, 5, 0.1, 500, 500)
# Field of view is around 340º for pigeons as saind in the source:
# https://www.quora.com/What-does-a-pigeons-field-of-view-look-like

[ 1.08706834  4.51189361]
[-3.65338528  3.66514555]
[ 2.49896089  4.28852381]
[ 1.84641426  4.42718857]
[ 4.54636566  0.34439434]
[ 1.21937334  4.5013348 ]
[ 3.25822137  3.97160972]
[-1.80015395  4.43426367]
[ 4.05779307  3.1031291 ]
[ 3.71482213  3.60147052]
[ 1.24608895  4.1677087 ]
[-3.78272143  3.79195325]
[ 2.7404409   4.17432155]
[ 2.07344543  4.19273409]
[ 4.14367764  0.39900619]
[ 1.39362665  4.17314791]
[ 3.45185864  4.01358821]
[-2.02450316  4.19202965]
[ 4.06693555  3.31335658]
[ 3.83045589  3.74146807]
[ 1.41990042  3.87811296]
[-3.91626925  3.92351623]
[ 2.9725727   4.11294297]
[ 2.30333607  4.02136139]
[ 3.78026376  0.46193075]
[ 1.58164449  3.9022408 ]
[ 3.63724099  4.0789893 ]
[-2.25270384  4.01294887]
[ 4.10653391  3.51303052]
[ 3.95326919  3.88341069]
[ 1.60512409  3.64368743]
[-4.05424371  4.05993258]
[ 3.19320251  4.09558675]
[ 2.52962246  3.90778654]
[ 3.45394386  0.53414069]
[ 1.77890921  3.68869148]
[ 3.81643169  4.16356488]
[-2.47819093  3.89217436]
[ 4.17077827

[ 6.95434289  6.95560665]
[-11.78596394  11.78596707]
[ 10.49447629  10.49497434]
[ 9.10487432  9.1056602 ]
[ 3.80034976  3.79764283]
[ 7.38024596  7.38140699]
[ 11.57849881  11.57869027]
[-8.99172954  8.99253798]
[ 11.41547808  11.41522031]
[ 11.78078512  11.78075492]
[ 7.19790285  7.1988949 ]
[-12.19847307  12.19847553]
[ 10.86184604  10.86223639]
[ 9.42364315  9.42426007]
[ 3.93302351  3.93089858]
[ 7.63869886  7.63961089]
[ 11.9837702  11.9839205]
[-9.30654113  9.30717575]
[ 11.81498759  11.81478524]
[ 12.19310883  12.19308511]
[ 7.44995345  7.45073221]
[-12.62541993  12.62542186]
[ 11.2420602   11.24236607]
[ 9.75354777  9.75403205]
[ 4.07041365  4.06874557]
[ 7.90616656  7.90688306]
[ 12.40322095  12.40333893]
[-9.63234939  9.63284757]
[ 12.22848686  12.22832802]
[ 12.61986467  12.61984606]
[ 7.71079916  7.71141048]
[-13.06730987  13.06731139]
[ 11.63557122  11.63581082]
[ 10.09498248  10.09536264]
[ 4.21266957  4.21136013]
[ 8.18297127  8.18353422]
[ 12.83734843  12.83744104]
[-

[-36.67724521  36.67724521]
[ 32.65899115  32.65899128]
[ 28.33506665  28.33506691]
[ 11.82225707  11.82225615]
[ 22.96869516  22.96869559]
[ 36.03191869  36.03191876]
[-27.98300171  27.98300199]
[ 35.52390866  35.52390857]
[ 36.66107717  36.66107716]
[ 22.40100209  22.40100243]
[-37.96094879  37.96094879]
[ 33.80205586  33.80205596]
[ 29.32679401  29.32679422]
[ 12.23603595  12.23603523]
[ 23.77259954  23.77259988]
[ 37.29303585  37.2930359 ]
[-28.96240681  28.96240702]
[ 36.76724545  36.76724538]
[ 37.94421487  37.94421486]
[ 23.18503721  23.18503747]
[-39.289582  39.289582]
[ 34.98512783  34.98512791]
[ 30.35323183  30.35323199]
[ 12.66429712  12.66429655]
[ 24.60464057  24.60464083]
[ 38.59829211  38.59829215]
[-29.97609107  29.97609124]
[ 38.05409903  38.05409898]
[ 39.27226239  39.27226238]
[ 23.99651354  23.99651375]
[-40.66471737  40.66471737]
[ 36.20960731  36.20960738]
[ 31.41559497  31.41559509]
[ 13.10754745  13.107547  ]
[ 25.46580302  25.46580323]
[ 39.94923234  39.949232

[ 69.71066147  69.71080358]
[-118.1328015   118.13302306]
[ 105.19004937  105.19004937]
[ 91.26278886  91.2625202 ]
[ 38.07783737  38.07783737]
[ 73.97895997  73.97895994]
[ 116.05382827  116.05368746]
[-90.12936939  90.12936956]
[ 114.41752424  114.41752398]
[ 118.0802091   118.08020914]
[ 72.15055065  72.15066371]
[-122.26747725  122.26765117]
[ 108.8717011  108.8717011]
[ 94.45695289  94.45674199]
[ 39.41056168  39.41056168]
[ 76.56822405  76.56822387]
[ 120.11569639  120.11558435]
[-93.28389734  93.28389747]
[ 118.42213756  118.42213735]
[ 122.21301594  122.21301612]
[ 74.67583363  74.67592271]
[-126.54686069  126.54699723]
[ 112.68221064  112.68221064]
[ 97.76291988  97.76275432]
[ 40.78993134  40.78993134]
[ 79.24811428  79.24811325]
[ 124.31973218  124.31964391]
[-96.54883376  96.54883387]
[ 122.56691235  122.56691219]
[ 126.49046911  126.49047014]
[ 77.28949885  77.28956884]
[-130.97601789  130.97612506]
[ 116.62608801  116.62608801]
[ 101.18460138  101.18447142]
[ 42.21757894 

[ 356.06074812  356.06072955]
[ 367.45855999  367.45856661]
[ 224.52852921  224.52852971]
[-380.4896124   380.48961227]
[ 338.80189841  338.80201036]
[ 293.94426337  293.94427789]
[ 122.64340385  122.64340388]
[ 238.27614893  238.27614365]
[ 373.79294684  373.79294673]
[-290.29371442  290.29390544]
[ 368.52287198  368.5228574 ]
[ 380.31961041  380.31961561]
[ 232.38702779  232.38702819]
[-393.80674882  393.80674871]
[ 350.65997885  350.66006673]
[ 304.23228059  304.23234538]
[ 126.93592299  126.93592301]
[ 246.61581348  246.61580934]
[ 386.87569997  386.87569988]
[-300.45401831  300.45416825]
[ 381.4212045   381.42113966]
[ 393.63079743  393.63080151]
[ 240.52057382  240.52057413]
[-407.58998501  407.58998493]
[ 362.93308909  362.93315808]
[ 314.88040708  314.88047095]
[ 131.3786803   131.37868031]
[ 255.24736644  255.24736319]
[ 400.41634946  400.41634939]
[-310.96992768  310.97004539]
[ 394.77094998  394.77088607]
[ 407.40787585  407.40787905]
[ 248.93879394  248.93879418]
[-421.8556

...

[  4.95436296e+42   4.95436296e+42]
[  3.02726951e+42   3.02726951e+42]
[ -5.13006262e+42   5.13006262e+42]
[  4.56799299e+42   4.56799299e+42]
[  3.96318666e+42   3.96318666e+42]
[  1.65357424e+42   1.65357424e+42]
[  3.21262470e+42   3.21262470e+42]
[  5.03976928e+42   5.03976928e+42]
[ -3.91396734e+42   3.91396734e+42]
[  4.96871390e+42   4.96871390e+42]
[  5.12776567e+42   5.12776567e+42]
[  3.13322394e+42   3.13322394e+42]
[ -5.30961481e+42   5.30961481e+42]
[  4.72787275e+42   4.72787275e+42]
[  4.10189820e+42   4.10189820e+42]
[  1.71144934e+42   1.71144934e+42]
[  3.32506657e+42   3.32506657e+42]
[  5.21616121e+42   5.21616121e+42]
[ -4.05095619e+42   4.05095619e+42]
[  5.14261889e+42   5.14261889e+42]
[  5.30723747e+42   5.30723747e+42]
[  3.24288678e+42   3.24288678e+42]
[ -5.49545133e+42   5.49545133e+42]
[  4.89334830e+42   4.89334830e+42]
[  4.24546463e+42   4.24546463e+42]
[  1.77135006e+42   1.77135006e+42]
[  3.44144390e+42   3.44144390e+42]
[  5.39872685e+42   5.398726

[  1.04448854e+43   1.04448854e+43]
[ -1.77000812e+43   1.77000812e+43]
[  1.57607914e+43   1.57607914e+43]
[  1.36740486e+43   1.36740486e+43]
[  5.70527116e+42   5.70527116e+42]
[  1.10844101e+43   1.10844101e+43]
[  1.73885452e+43   1.73885452e+43]
[ -1.35042289e+43   1.35042289e+43]
[  1.71433852e+43   1.71433852e+43]
[  1.76921561e+43   1.76921561e+43]
[  1.08104564e+43   1.08104564e+43]
[ -1.83195840e+43   1.83195840e+43]
[  1.63124191e+43   1.63124191e+43]
[  1.41526403e+43   1.41526403e+43]
[  5.90495565e+42   5.90495565e+42]
[  1.14723645e+43   1.14723645e+43]
[  1.79971443e+43   1.79971443e+43]
[ -1.39768769e+43   1.39768769e+43]
[  1.77434037e+43   1.77434037e+43]
[  1.83113816e+43   1.83113816e+43]
[  1.11888224e+43   1.11888224e+43]
[ -1.89607695e+43   1.89607695e+43]
[  1.68833538e+43   1.68833538e+43]
[  1.46479828e+43   1.46479828e+43]
[  6.11162909e+42   6.11162909e+42]
[  1.18738972e+43   1.18738972e+43]
[  1.86270443e+43   1.86270443e+43]
[ -1.44660676e+43   1.446606

[  5.41391100e+43   5.41391100e+43]
[  2.25886503e+43   2.25886503e+43]
[  4.38860586e+43   4.38860586e+43]
[  6.88457665e+43   6.88457665e+43]
[ -5.34667494e+43   5.34667494e+43]
[  6.78751145e+43   6.78751145e+43]
[  7.00478411e+43   7.00478411e+43]
[  4.28014045e+43   4.28014045e+43]
[ -7.25319912e+43   7.25319912e+43]
[  6.45851040e+43   6.45851040e+43]
[  5.60339789e+43   5.60339789e+43]
[  2.33792530e+43   2.33792530e+43]
[  4.54220706e+43   4.54220706e+43]
[  7.12553683e+43   7.12553683e+43]
[ -5.53380856e+43   5.53380856e+43]
[  7.02507435e+43   7.02507435e+43]
[  7.24995155e+43   7.24995155e+43]
[  4.42994537e+43   4.42994537e+43]
[ -7.50706109e+43   7.50706109e+43]
[  6.68455826e+43   6.68455826e+43]
[  5.79951681e+43   5.79951681e+43]
[  2.41975269e+43   2.41975269e+43]
[  4.70118431e+43   4.70118431e+43]
[  7.37493062e+43   7.37493062e+43]
[ -5.72749186e+43   5.72749186e+43]
[  7.27095195e+43   7.27095195e+43]
[  7.50369985e+43   7.50369985e+43]
[  4.58499346e+43   4.584993

[  2.25614313e+44   2.25614313e+44]
[  1.37857346e+44   1.37857346e+44]
[ -2.33615413e+44   2.33615413e+44]
[  2.08019600e+44   2.08019600e+44]
[  1.80477620e+44   1.80477620e+44]
[  7.53013088e+43   7.53013088e+43]
[  1.46298146e+44   1.46298146e+44]
[  2.29503589e+44   2.29503589e+44]
[ -1.78236245e+44   1.78236245e+44]
[  2.26267833e+44   2.26267833e+44]
[  2.33510814e+44   2.33510814e+44]
[  1.42682353e+44   1.42682353e+44]
[ -2.41791953e+44   2.41791953e+44]
[  2.15300286e+44   2.15300286e+44]
[  1.86794337e+44   1.86794337e+44]
[  7.79368546e+43   7.79368546e+43]
[  1.51418581e+44   1.51418581e+44]
[  2.37536215e+44   2.37536215e+44]
[ -1.84474514e+44   1.84474514e+44]
[  2.34187207e+44   2.34187207e+44]
[  2.41683692e+44   2.41683692e+44]
[  1.47676236e+44   1.47676236e+44]
[ -2.50254671e+44   2.50254671e+44]
[  2.22835796e+44   2.22835796e+44]
[  1.93332138e+44   1.93332138e+44]
[  8.06646445e+43   8.06646445e+43]
[  1.56718231e+44   1.56718231e+44]
[  2.45849983e+44   2.458499

[  8.63058610e+44   8.63058610e+44]
[  5.27355592e+44   5.27355592e+44]
[ -8.93665794e+44   8.93665794e+44]
[  7.95752292e+44   7.95752292e+44]
[  6.90393982e+44   6.90393982e+44]
[  2.88055496e+44   2.88055496e+44]
[  5.59644788e+44   5.59644788e+44]
[  8.77936539e+44   8.77936539e+44]
[ -6.81819889e+44   6.81819889e+44]
[  8.65558568e+44   8.65558568e+44]
[  8.93265661e+44   8.93265661e+44]
[  5.45813038e+44   5.45813038e+44]
[ -9.24944096e+44   9.24944096e+44]
[  8.23603622e+44   8.23603622e+44]
[  7.14557771e+44   7.14557771e+44]
[  2.98137439e+44   2.98137439e+44]
[  5.79232355e+44   5.79232355e+44]
[  9.08664318e+44   9.08664318e+44]
[ -7.05683586e+44   7.05683586e+44]
[  8.95853118e+44   8.95853118e+44]
[  9.24529959e+44   9.24529959e+44]
[  5.64916494e+44   5.64916494e+44]
[ -9.57317140e+44   9.57317140e+44]
[  8.52429749e+44   8.52429749e+44]
[  7.39567293e+44   7.39567293e+44]
[  3.08572249e+44   3.08572249e+44]
[  5.99505488e+44   5.99505488e+44]
[  9.40467569e+44   9.404675